# II. Programming and critical analysis

Consider the winequality-red.csv dataset (available at the webpage) where the goal is to estimate
the quality (sensory appreciation) of a wine based on physicochemical inputs.

Using a 80-20 training-test split with a fixed seed (random_state=0), you are asked to learn MLP
regressors to answer the following questions.

Given their stochastic behavior, average the performance of each MLP from 10 runs
(for reproducibility consider seeding the MLPs with random_state ∈ {1. .10}).

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt

In [3]:
# Reading the CSV file
df = pd.read_csv("winequality-red.csv", delimiter=";")

# Some of the sklearn facilities require input data to be separated from output data
X = df.drop("quality", axis=1)
y = df["quality"]

In [4]:
np.random.seed(0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

num_runs = 10

## 1)
Learn a MLP regressor with 2 hidden layers of size 10, rectifier linear unit activation on all nodes, and early stopping with 20% of training data set aside for validation. All remaining parameters (e.g., loss, batch size, regularization term, solver) should be set as
default. Plot the distribution of the residues (in absolute value) using a histogram.

In [5]:
residuals = []

for random_state in range(1, num_runs + 1):
    mlp_regressor = MLPRegressor(hidden_layer_sizes=(10, 10), 
                                 activation='relu', 
                                 early_stopping=True, 
                                 validation_fraction=0.2, 
                                 random_state=random_state)
    
    mlp_regressor.fit(X_train, y_train)
    y_predictions = mlp_regressor.predict(X_test)
    residuals.append(np.abs(y_test - y_predictions))


all_residuals = np.concatenate(residuals)

plt.figure(figsize=(8, 6))
plt.hist(all_residuals, edgecolor='black')
plt.xlabel('Residuals (Absolute Value)')
plt.ylabel('Frequency')
plt.title('Distribution of Residuals')
plt.grid(True)
plt.show()


/home/ana/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ana/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ana/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ana/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warning

## 2)
Since we are in the presence of a integer regression task, a recommended trick is to round and bound estimates. Assess the impact of these operations on the MAE of the MLP learnt in previous question.

In [6]:
rounded_bounded_mae_values = []
mae_values = []

for random_state in range(1, num_runs + 1):
    mlp_regressor = MLPRegressor(hidden_layer_sizes=(10, 10), 
                                 activation='relu', 
                                 early_stopping=True, 
                                 validation_fraction=0.2, 
                                 random_state=random_state)
    
    mlp_regressor.fit(X_train, y_train)
    y_predictions = mlp_regressor.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_predictions)
    mae_values.append(mae)

    rounded_bounded_mae = mean_absolute_error(y_test, np.round(np.clip(y_predictions, 3, 8)))
    rounded_bounded_mae_values.append(rounded_bounded_mae)

average_mae = np.mean(mae_values)
average_rounded_bounded_mae = np.mean(rounded_bounded_mae_values)

print(f"Average MAE: {average_mae}")
print(f"Average MAE after rounding and bounding: {average_rounded_bounded_mae}")


/home/ana/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ana/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ana/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ana/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warning

Average MAE: 0.5097171955009514
Average MAE after rounding and bounding: 0.43875000000000003


/home/ana/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


## 3) 
Similarly assess the impact on RMSE from replacing early stopping by a well-defined number of iterations in {20,50,100,200} (where one iteration corresponds to a batch).

In [7]:
num_iterations = [20, 50, 100, 200]

average_rmse = {}

for num_iteraction in num_iterations:
    rmse_results = []
    for random_state in range(1, num_runs + 1):
        mlp_regressor = MLPRegressor(hidden_layer_sizes=(10, 10), 
                                     activation='relu', 
                                     max_iter=num_iteraction, 
                                     validation_fraction=0.2, 
                                     random_state=random_state)
        
        mlp_regressor.fit(X_train, y_train)
        y_predictions = mlp_regressor.predict(X_test)
        rmse = sqrt(mean_squared_error(y_test, y_predictions))
        rmse_results.append(rmse)
    average_rmse[num_iteraction] = np.mean(rmse_results)

for i, iterations in enumerate(num_iterations):
    print(f'RMSE for {iterations} iterations: {average_rmse[iterations]}')

/home/ana/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ana/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ana/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/ana/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.wa

RMSE for 20 iterations: 1.4039789509925442
RMSE for 50 iterations: 0.7996073631460568
RMSE for 100 iterations: 0.6940361469112143
RMSE for 200 iterations: 0.6554543932216474


/home/ana/.pyenv/versions/3.10.12/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


## 4) Critically comment the results obtained in previous question, hypothesizing at least one reason why early stopping favors and/or worsens performance.

In general, early stopping favors performance by preventing overfitting, where the model has over-adapted to the training data, essentially memorizing it rather than actually learning its underlying patterns, thus losing its ability to generalize.

However, as we can observe from the results obtained in the previous question, it is crucial to balance the number of iterations to avoid worsened performance.
The RMSE value consistently decreases as the number of iterations increases, indicating that too few iterations lead to underfitting.